# Rough

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib widget

In [29]:
import pandas as pd

import seaborn as sns

import numpy as np
import matplotlib.pyplot as plt

from xgboost import XGBRegressor
import optuna

from sklearn.metrics import mean_squared_error

import warnings

from poly2.utils import (
    get_dispersal_kernel, trait_vec, load_train_test_data,
    get_best_model, get_model_cv_score, HyperparamsObj,
    get_best_params, 
)
from cluster.hyper import train_test_scores, run_optuna
from poly2.config import Config

In [30]:
warnings.simplefilter(action='ignore', category=FutureWarning)

## Which model?

In [3]:
model = 'all'

## SHAP stuff

In [77]:
0.11017603330089432

0.11017603330089432

In [88]:
df1 = pd.DataFrame(best_pars, index=[0])

In [89]:
df2 = pd.DataFrame(
    dict(
    model=1,
    default_cv_score=2,
    ), index=[0]
)

In [92]:
pd.concat([df1, df2], axis=1)

,max_depth,n_estimators,learning_rate,subsample,colsample_bytree,model,default_cv_score
0,10,1740,0.019909,0.546807,0.887985,1,2


In [91]:
df2

,model,default_cv_score
0,1,2


In [84]:
(
    pd.read_csv(f'../outputs/scores/{model}.csv')
    .loc[:, ['best_cv_score']]
    # .loc[lambda x: x.best_cv_score>0.1101]
    # .loc[lambda x: x.best_cv_score<0.1105]
)

,best_cv_score
2,0.110323
22,0.110346
118,0.110283
124,0.110490
135,0.110310
152,0.110398


In [42]:
hps = pd.read_csv(f'../outputs/hyperparams/best/{model}.csv')
hps

,model,default_cv_score,best_cv_score,rmse_test,rmse_train,rmse_test_def,number,max_depth,n_estimators,learning_rate,subsample,colsample_bytree
0,all,0.131867,0.109074,0.107335,0.042964,0.12707,3,10,1740,0.019909,0.546807,0.887985


In [58]:
bps = get_best_params(model)

In [63]:
del bps['tree_method']

In [64]:
rmse_train, rmse_test, rmse_test_def = train_test_scores(
    bps,
    X_cv,
    y_cv,
    X_test,
    y_test,
)

In [65]:
rmse_train, rmse_test, rmse_test_def

(0.0431882810074089, 0.10774565717579333, 0.12707018807068118)

In [61]:
rmse_train, rmse_test, rmse_test_def

(0.03718551081030404, 0.10623304379219752, 0.12707018807068118)

# Try re-running optuna

In [93]:
np.random.seed(99)

optuna.logging.set_verbosity(3)

obj = HyperparamsObj(X_cv, y_cv)

sampler = optuna.samplers.RandomSampler(seed=99)


study = optuna.create_study(sampler=sampler)

study.optimize(obj, n_trials=1)

best_pars = study.best_params
best_value = study.best_value
best_number = study.best_trial.number

[I 2022-09-13 10:12:13,489] A new study created in memory with name: no-name-24bdcf42-cfa0-40de-a9d5-9a94a6a12c65
5it [02:16, 27.29s/it]
[I 2022-09-13 10:14:30,038] Trial 0 finished with value: 0.1226907805611435 and parameters: {'max_depth': 15, 'n_estimators': 129, 'learning_rate': 0.11087959763246974, 'subsample': 0.515723193813149, 'colsample_bytree': 0.8848299780189086}. Best is trial 0 with value: 0.1226907805611435.


In [98]:
best_pars = study.best_params
study.best_params

{'max_depth': 15,
 'n_estimators': 129,
 'learning_rate': 0.11087959763246974,
 'subsample': 0.515723193813149,
 'colsample_bytree': 0.8848299780189086}

In [99]:
best_pars['tree_method'] = 'hist'

In [73]:
study.best_value

0.11017603330089432

In [74]:
study.best_trial.number

3

In [75]:
best_value

0.11017603330089432

# old

In [ ]:
best_model = get_best_model(model)

In [47]:
X_cv, y_cv, X_test, y_test = load_train_test_data(model)

Below cell takes around `1 minute`

In [48]:
%%time
best_model.fit(X_cv, y_cv)

Wall time: 58.3 s


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8879850476692774,
             enable_categorical=False, gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.0199085136730978,
             max_delta_step=0, max_depth=10, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=1740, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=0.5468065466488763,
             tree_method='hist', validate_parameters=1, verbosity=None)

In [49]:
yptest = best_model.predict(X_test)

In [50]:
rmse_test = mean_squared_error(yptest, y_test, squared=False)
rmse_test

0.10623304379219752

We are expecting a value of `0.1073346696102688` but the setup didn't use `tree_method=hist`.

We got a value of `0.10623304379219752` when seed was 99, same for 200

So we are all good (??)

In [38]:
yptrain = best_model.predict(X_cv)

In [39]:
rmse_train = mean_squared_error(yptrain, y_cv, squared=False)
rmse_train

0.03718551081030404

## Check model cv score

In [23]:
best_params = (
    pd.read_csv(f'../outputs/hyperparams/best/{model}.csv')
    .loc[:, [
        'max_depth',
        'n_estimators',
        'learning_rate',
        'subsample',
        'colsample_bytree',
    ]]
    .iloc[0]
    .to_dict()
)

best_params['n_estimators'] = int(best_params['n_estimators'])
best_params['max_depth'] = int(best_params['max_depth'])
best_params['tree_method'] = 'hist'

best_params

{'max_depth': 10,
 'n_estimators': 1740,
 'learning_rate': 0.0199085136730978,
 'subsample': 0.5468065466488763,
 'colsample_bytree': 0.8879850476692774,
 'tree_method': 'hist'}

In [25]:
model_cv_score = get_model_cv_score(X_cv, y_cv, best_params)

0it [00:00, ?it/s]C:\Users\user\anaconda3\envs\poly2\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
1it [00:52, 52.74s/it]C:\Users\user\anaconda3\envs\poly2\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
2it [01:45, 52.72s/it]C:\Users\user\anaconda3\envs\poly2\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
3it [02:38, 52.69s/it]C:\Users\user\anaconda3\envs\poly2\lib\s

0.11017603330089432

`0.11017603330089432`

## Check default model

In [32]:
default_model = XGBRegressor().fit(X_cv, y_cv)

yp_def = default_model.predict(X_test)

rmse_def_test = mean_squared_error(yp_def, y_test, squared=False)

## Result for paper (table)

In [41]:
print(f'{rmse_test=:.5f}\n{rmse_train=:.5f}\n{rmse_def_test=:.5f}')

rmse_test=0.10623
rmse_train=0.03719
rmse_def_test=0.12707


In [43]:
hps

,model,default_cv_score,best_cv_score,rmse_test,rmse_train,rmse_test_def,number,max_depth,n_estimators,learning_rate,subsample,colsample_bytree
0,all,0.131867,0.109074,0.107335,0.042964,0.12707,3,10,1740,0.019909,0.546807,0.887985


In [44]:
best_params

{'max_depth': 10,
 'n_estimators': 1740,
 'learning_rate': 0.0199085136730978,
 'subsample': 0.5468065466488763,
 'colsample_bytree': 0.8879850476692774,
 'tree_method': 'hist'}